In [1]:
!git clone https://github.com/moaaztaha/Arabic-English-Translation-Transformers

Cloning into 'Arabic-English-Translation-Transformers'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 8 (delta 0), reused 8 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), done.


In [3]:
# modules
import random
import string
import re
from pathlib import Path
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

### Data Preprocessing 

In [11]:
import pandas as pd
# df = pd.read_csv("/content/Arabic-English-Translation-Transformers/ara_eng.txt",delimiter="\t",names=["eng","ar"])
# df.head()

df_ar = pd.read_table('data/ara/test.en_ref.ar', names=['arabic'], encoding='utf-8')
df_en = pd.read_table('data/eng/ac-test.en', names=['english'])

In [12]:
print(df_ar.shape)
df_ar.head()

(3964, 1)


,arabic
0,مجلس الجماعة الاقتصادية الأوروبية
1,حيث أن اعتماد سياسة نقل مشتركة تنطوي من بين أم...
2,المادة 1
3,3. لا تخضع أنواع النقل المدرجة في الملحق الثان...
4,تبلغ الدول الأعضاء المفوضية الأوروبية بالتدابي...


In [13]:
print(df_en.shape)
df_en.head()

(4046, 1)


,english
0,"THE COUNCIL OF THE EUROPEAN ECONOMIC COMMUNITY,"
1,Whereas the adoption of a common transport pol...
2,Article 1
3,3. The types of carriage listed in Annex II sh...
4,Member States shall inform the Commission of t...


In [14]:
df_ar.tail()

,arabic
3959,باعتبار الطلب الذي تقدمت به لوكسمبورغ في 25 تم...
3960,(2) يجب أن يتم منح هذه الاستثناءات إلى النمسا ...
3961,المادة 1
3962,(ب) تمنح فرنسا الاستثناءات للحصول على النتائج ...
3963,بعد انتهاء الفترة الانتقالية، تقوم النمسا وفرن...


In [15]:
df_en.tail()

,english
4041,Having regard to the request made by Luxembour...
4042,"(2) Such derogations should be granted, at the..."
4043,Article 1
4044,(b) France is granted derogations for the prod...
4045,"After expiry of the transitional period, Austr..."


In [37]:
for i in range(df_en.shape[0]):
  if abs(len(df_en.english.iloc[-i].split()) - len(df_ar.arabic.iloc[-i].split())) > 30:
    print(df_en.english.iloc[-i]) 
    print(df_ar.arabic.iloc[-i])
    print(i)
    print("_"*50)
    break

Article 6a
2. All ro-ro passenger ships of Classes A and B, the keel of which is laid or which are at a similar stage of construction before 1 October 2004 shall comply with Articles 6, 8 and 9 of Directive 2003/25/EC by 1 October 2010, unless they are phased out on that date or on a later date on which they reach the age of 30 years but in any case not later than 1 October 2015.
1. Member States shall ensure that appropriate measures are taken, based, where practicable, on the guidelines in Annex III to enable persons with reduced mobility to have safe access to all passenger ships of Classes A, B, C and D and to all high-speed passenger craft, used for public transport, the keel of which is laid or which are at a similar stage of construction on or after 1 October 2004.
Member States shall draw up a national action plan on how the guidelines shall be applied to such ships and craft. They shall forward that plan to the Commission not later than 17 May 2005.
Article 2
Member States sha

In [46]:
a, b = 65, 75
df_en[a: b]

,english
65,Article 11
66,Article 12
67,"THE COUNCIL OF THE EUROPEAN COMMUNITIES,"
68,Having regard to the Opinion of the European P...
69,"Whereas it has become apparent, in the course ..."
70,Whereas certain of the abovementioned Directiv...
71,HAS ADOPTED THIS DIRECTIVE:
72,G. Small EEC packages : packages containing th...
73,REGULATION (EEC) No 2777/75 OF THE COUNCIL of ...
74,Having regard to the proposal from the Commiss...


In [47]:
df_ar[a: b]

,arabic
65,المادة 11
66,المادة 12
67,مجلس الجماعات الأوروبية،
68,مع الأخذ في الاعتبار رأي البرلمان الأوروبي (1)؛
69,حيث أنه أصبح واضحاً، في سياق تطبيق التوجيهات، ...
70,حيث أن بعض التوجيهات السابق ذكرها تنص على أنه ...
71,قد اعتمدت هذا التوجيه:
72,رزم الجماعة الاقتصادية الأوروبية الصغيرة: تحتو...
73,2. تطلب الدول الأعضاء، ألا يتم إغلاق الرزم ثان...
74,4. يحل ما يلي محل القسم التمهيدي للمادة 11 (1):


In [48]:
c = 72
df_ar.iloc[c].arabic

'رزم الجماعة الاقتصادية الأوروبية الصغيرة: تحتوي الرزم على البذور التالية المرخص لها – وهي البذور المفردة أو البذور الدقيقة التي لا تتعدى الـ100 000 عنقود أو حبة أو التي لا يتخطى وزنها الصافي 2.5 كيلوغرام باستثناء، عند الإقتضاء، المبيدات الحبيبية أو مواد التحبيب أو إضافات صلبة أخرى؛'

In [50]:
df_ar.iloc[c+1].arabic

'2. تطلب الدول الأعضاء، ألا يتم إغلاق الرزم ثانيةً في حالة واحدة أو عدة حالات إلا إذا جرى ذلك رسمياً، وذلك باستثناء رزم الجماعة الاقتصادية الأوروبية الصغيرة. إذا تم إغلاق الرزم مجدداً، يتم ذكر إعادة الإغلاق وتاريخها والسلطة المسؤولة عنها على الشارة المطلوبة بموجب المادة 11 (1).'

In [51]:
df_ar.iloc[c+2].arabic

'4. يحل ما يلي محل القسم التمهيدي للمادة 11 (1):'

In [52]:
df_ar.iloc[c+3].arabic

'(ب) توفير استثناءات للفقرة 1 في حال الرزم الصغيرة للبذور الأساسية التي ذُكر عليها: "نُقلت للتسويق في ... (الدول العضو المعنية) فقط".'

In [53]:
df_ar.iloc[c+4].arabic

'نظام (اللجنة الإقتصادية الأوروبية) رقم 2777/75 للمجلس المؤرخ في 29 أكتوبر/تشرين الثاني 1975 بشأن التنظيم المشترك للسوق في مجال لحوم الدواجن'

In [49]:
df_en.iloc[c].english

'G. Small EEC packages : packages containing the following certified seeds: - monogerm or precision seed : not exceeding 100 000 clusters or grains or a net weight of 2.5 kg excluding, where appropriate, granulated pesticides, pelleting substances or other solid additives;\n2. Member States shall require that, except in the case of small EEC packages, packages shall not be resealed on one or more occasions unless this is done officially. If packages are resealed, the fact of resealing, the date of resealing and the authority responsible therefor shall be stated on the label required under Article 11 (1).\n4. The following is to be substituted for the introductory section of Article 11 (1):\n(b) provide for exceptions to paragraph 1 in the case of small packages of basic seed where they are marked : "Passed for marketing in ... (Member State concerned) only"."'

In [17]:
for i in range(df_en.shape[0]):
  if len(df_en.english.iloc[-i].split()) > 200:
    print(df_en.english.iloc[-i]) 
    print(df_ar.arabic.iloc[-i])
    print(i)
    print("_"*50)

Article 6a
2. All ro-ro passenger ships of Classes A and B, the keel of which is laid or which are at a similar stage of construction before 1 October 2004 shall comply with Articles 6, 8 and 9 of Directive 2003/25/EC by 1 October 2010, unless they are phased out on that date or on a later date on which they reach the age of 30 years but in any case not later than 1 October 2015.
1. Member States shall ensure that appropriate measures are taken, based, where practicable, on the guidelines in Annex III to enable persons with reduced mobility to have safe access to all passenger ships of Classes A, B, C and D and to all high-speed passenger craft, used for public transport, the keel of which is laid or which are at a similar stage of construction on or after 1 October 2004.
Member States shall draw up a national action plan on how the guidelines shall be applied to such ships and craft. They shall forward that plan to the Commission not later than 17 May 2005.
Article 2
Member States sha

In [28]:
df_en.iloc[410:420]

,english
410,- submit once per month in a standardized form...
411,Article 7
412,- has sold either some or all of his olive pro...
413,1. Before submitting the aid application each ...
414,2. Producer organizations shall forward the fi...
415,- where the particulars of the crop declaratio...
416,"However, the producer organizations comprising..."
417,- shall coordinate the activities of the organ...
418,Article 11
419,(b) the balance shall be paid to all the produ...


In [30]:
df_ar.iloc[395:405]

,arabic
395,- تقدم مرة واحدة في الشهر في شكل موحد مناسب لم...
396,المادة 7
397,- باعت بعض أو كامل إنتاجها من الزيتون، و
398,1. قبل تقديم طلب الدعم، يتوجب على كل منظمة منت...
399,2. يتوجب على المنظمات المنتجة تقديم ملفات أعضا...
400,- عندما لا تتوافق تفاصيل تصريح المحاصيل مع الو...
401,ولكن، يجب أن تكون المنظمات المنتجة التي تشمل ش...
402,- يتوجب عليها تنسيق أعمال المنظمات المؤلفة منه...
403,المادة 11
404,(ب) يجب تسديد الميزانية لكل المنظمات المنتجة ...


In [19]:
df_en[df_en['english'] == 'Article 2']

,english
31,Article 2
36,Article 2
46,Article 2
82,Article 2
154,Article 2
...,...
3628,Article 2
3639,Article 2
3925,Article 2
3999,Article 2


In [23]:
df_ar[df_ar['arabic'] == "المادة 2"]

,arabic
31,المادة 2
36,المادة 2
46,المادة 2
85,المادة 2
157,المادة 2
...,...
3815,المادة 2
3856,المادة 2
3863,المادة 2
3917,المادة 2


In [255]:
text_pairs = []
for idx, row in df.iterrows():
    eng, ar = row['eng'], row['ar']
    ar = "[start] " + ar + " [end]"
    text_pairs.append((eng, ar))

In [256]:
for _ in range(2):
    print(random.choice(text_pairs))

('during the closing session of the new media workshop i led recently in alexandria egypt practically everybody paid tribute to shahinaz abdelsalam better known in the blogosphere as wahda masrya an egyptian girl the lone alexandria native and one of the few experienced bloggers among the group shahinaz became for most of the participants a symbol of courage and deep commitment to the cause of human rights and of freedom expression.', '[start] خلال الجلسة النهايية من ورشة الاعلام الجديدة التي نظمتها في الاسكندرية مصر عمد الجميع للتعبير عن تقديرهم لشهيناز عبد السلام المعروفة ب واحدة مصرية عربي في عالم المدونات تلك الفتاة الوحيدة المتاصلة من الاسكندرية ومن بين قلة من المدونين المصريين المخضرمين في المجموعة باتت شاهيناز بالنسبة للعديد من المشاركين في الورشة رمزا للشجاعة والاخلاص لقضيتي حقوق الانسان وحرية التعبير [end]')
('He is a naughty boy.', '[start] هو ولد مشاغب. [end]')


In [257]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) -  num_val_samples
train_pairs = text_pairs[: num_train_samples]
val_pairs = text_pairs[num_train_samples: num_train_samples + num_val_samples]

In [258]:
print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")

24638 total pairs
20943 training pairs
3695 validation pairs


#### Vectorizing the text data 

In [265]:
strip_chars = string.punctuation
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")


vocab_size = 20000
sequence_length = 150
batch_size = 128

def custom_standardization(input_string):
    return tf.strings.regex_replace(input_string, "[%s]" % re.escape(strip_chars), "")

eng_vectorization = TextVectorization(
            max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_length)

ar_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization)

eng_texts = [pair[0] for pair in text_pairs]
ar_texts = [pair[1] for pair in text_pairs]
eng_vectorization.adapt(eng_texts)
ar_vectorization.adapt(ar_texts)

In [270]:
def format_dataset(eng, ar):
    eng = eng_vectorization(eng)
    ar = ar_vectorization(ar)
    return ({"encoder_inputs": eng, "decoder_inputs": ar[:, :-1],}, ar[:, 1:])


def make_dataset(pairs):
    eng_texts, ar_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    ar_texts = list(ar_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, ar_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()

In [271]:
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [272]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (128, 150)
inputs["decoder_inputs"].shape: (128, 150)
targets.shape: (128, 150)


### Building the Model 

In [273]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):

      config = super().get_config().copy()
      config.update({
          'embed_dim': self.embed_dim,
          'dense_dim': self.dense_dim,
          'num_heads': self.num_heads,
      })
      return config

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
      
    def get_config(self):

      config = super().get_config().copy()
      config.update({
      'sequence_length': self.sequence_length,
      'vocab_size': self.vocab_size,
      'embed_dim': self.embed_dim,
      })
      return config

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True


    def get_config(self):

      config = super().get_config().copy()
      config.update({
      'embed_dim': self.embed_dim,
      'latent_dim': self.latent_dim,
      'num_heads': self.num_heads,
      })
      return config

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

In [274]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [275]:
googledrive_path = '/content/drive/MyDrive/Transformers/'

In [276]:
from keras import callbacks
early_stopping_cb = callbacks.EarlyStopping(monitor='val_accuracy', patience=4, verbose=True)
checkpoint_cb = callbacks.ModelCheckpoint(googledrive_path+'model.h5', monitor='val_accuracy', verbose=True, save_best_only=True)
tensorboard_callback = callbacks.TensorBoard(log_dir=googledrive_path+"logs")
cbs = [early_stopping_cb, checkpoint_cb, tensorboard_callback]

In [277]:
epochs = 30  # This should be at least 30 for convergence

transformer.summary()
transformer.compile(
    "adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=cbs)

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
positional_embedding_24 (Positi (None, None, 256)    5158400     encoder_inputs[0][0]             
__________________________________________________________________________________________________
decoder_inputs (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
transformer_encoder_12 (Transfo (None, None, 256)    3155456     positional_embedding_24[0][0]    
________________________________________________________________________________________

In [247]:
ar_vocab = ar_vectorization.get_vocabulary()
ar_index_lookup = dict(zip(range(len(ar_vocab)), ar_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = ar_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = ar_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


test_eng_texts = [pair[0] for pair in val_pairs]
for _ in range(30):
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequence(input_sentence)
    print(input_sentence, '\n', translated)
    print('*'*50)

لماذا يسعى الاوربيون الى طريق بحري الى الهند؟ 
 [start] why are not going to leave their seats to a blood [end]
**************************************************
دمتري اولتشانسكي الصورة من فيسبوك 
 [start] [UNK] on flickr photo by flickr user [end]
**************************************************
ولد فريق كومابا يوم من اغسطس اب بعد الحفلة السنوية للمنهج التعليمي مقدمة للموسيقى الفنزويلية وامريكا اللاتينية وخرج الفريق من بين الموسيقيين من هذا الفصل وايضا من هولاء الراغبين في لعب الموسيقى الفنزويلية طوال العام جاءت فكرة تدريس الموسيقى الفنزويلية في جامعة يابانية بعد سنة من عمل الاستاذ اشيباشي على نشر الموسيقى الفنزويلية بعيدا عن منشاها الاصلي 
 [start] a mother of the girl from august after the party went to the party of the brazilian artists from his
**************************************************
أنا أتتطلع لحفلة عيد القديسيين خاصتك. 
 [start] im looking forward to the christmas dinner [end]
**************************************************
بوليفيا تسمية مدون لرياسة المحكمة الان